## Data Pre-processing

### Download NCBI dataset from Huggingface and Re-create according to input method

In [1]:
!git clone https://github.com/isegura/NLP4RARE-CM-UC3M.git

Cloning into 'NLP4RARE-CM-UC3M'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 532 (delta 84), reused 103 (delta 58), pack-reused 379
Receiving objects: 100% (532/532), 7.92 MiB | 7.20 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [2]:
!pip install smart-open mendelai-brat-parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for mendelai-brat-parser: filename=mendelai_brat_parser-0.0.11-py3-none-any.whl size=4944 sha256=e1e9f2830d757e7a9a6f489c8b7b5da4ea80eb5b2f8dcf095194bd94094da8cf
  Stored in directory: /root/.cache/pip/wheels/90/a7/ff/138853d8196095fec56e0a97779a96d754b98f169c063beca3
Successfully built mendelai-brat-parser


In [ ]:
!cd /content/NLP4RARE-CM-UC3M/corpus && unzip train.zip

In [6]:
import os

base_path = '/content/NLP4RARE-CM-UC3M/corpus/train/'

all_files = [ fi for fi in os.listdir(base_path) if fi.endswith(".ann") or fi.endswith(".txt") ]
ann_files = [ fi for fi in all_files if not fi.endswith(".ann") ]
txt_files = [ fi for fi in all_files if not fi.endswith(".txt") ]

len(all_files) == len(ann_files)+len(txt_files)

True

In [7]:
from brat_parser import get_entities_relations_attributes_groups

def read_corpus(filename):
    sentence = {}
    txt = open(base_path+filename+'.txt')
    sentence['text'] = txt.read()
    entities, _, _, _ = get_entities_relations_attributes_groups(base_path+filename+'.ann')
    labels = []
    for k in entities.keys():
        if entities.get(k).text == sentence['text'][entities.get(k).span[0][0]:entities.get(k).span[0][1]]:
            labels.append([entities.get(k).span[0][0], entities.get(k).span[0][1], entities.get(k).type])
    sentence['label'] = labels
    return sentence

In [8]:
annotated = []
for idx,file in enumerate(ann_files):
    annotated.append(read_corpus(file.split(".")[0]))

In [11]:
corpus = annotated

### Preprocess to feed model

In [12]:
corpus[0]

{'text': 'Tongue cancers are oral cancers that are differentiated by their location in the mouth and on the tongue. If the cancer is on the forward portion of the tongue, it is known as a squamous cell cancer of the oral tongue. If the cancer is located towards the rear third of the tongue, it is known as a squamous cell cancer at the base of the tongue. Tongue cancer is most common in men over age 60.  It is rare in people, particularly women, under age 40. Examination of a sample of tissue from the site of the suspected cancer by a qualified pathologist is the key to diagnosis. MRI and/or CAT scans may be ordered to determine the location and size of the growth. This examination will also determine the stage of the disorder (how advanced it may be), which in turn, will help determine the method and pace of treatment.\n',
 'label': [[0, 14, 'RAREDISEASE'],
  [19, 31, 'DISEASE'],
  [113, 119, 'DISEASE'],
  [178, 217, 'RAREDISEASE'],
  [226, 232, 'DISEASE'],
  [299, 345, 'RAREDISEASE'],

In [13]:
# Collect all entites into on array
all_entities = []
for cor in corpus:
    entities = []
    for e in cor['label']:
        build_entity = ''
        for i in range(e[0], e[1]):
            build_entity += cor['text'][i]
        entities.append(build_entity)
    all_entities.append(entities)

In [14]:
count = 0
for en in all_entities:
    count += len(en)
print('Entity Count: ',count)

words = 0
for w in corpus:
    words += len(w['text'].split(' '))
print('Word Count: ', words)

Entity Count:  9751
Word Count:  115523


In [15]:
sentences = []
for doc in corpus:
    for s in doc['text'].split('.'):
        sentences.append(s)
len(sentences)

7408

In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
from nltk.corpus import stopwords

# Get a list of English stop words
stop_words = stopwords.words('english')

## Format documents according required input params

In [24]:
import re
useless_chars = ['(', ')', ':', '\n\n', '\n', '[', ']']
concat_sent_cfg = []
annotated_docs = []

# Select one item from the '''corpus'''
for idx,document in enumerate(corpus):
    sentences_cfg = []
    # Get an array of entities with starting and ending indexes as '''entities''' and '''text'''
    curr_entities = document['label']
    original_text = document['text']
    for uc in useless_chars:
        original_text = original_text.replace(uc, ' ')
    # Split the text from '''.''' and add the start and ending values from the original text
    for curr_sent in original_text.split('.'):
        if len(curr_sent) > 5:
          try:
            for sen in re.finditer(curr_sent, original_text):
              sentences_cfg.append({
                  'text': curr_sent,
                  'start': sen.start(),
                  'end': sen.end()
              })
          except:
            print('Skipping...'+ str(idx))
    # Check whether the tagged entity include in this sentence or not
    for new_sent in sentences_cfg:
        curr_labels = []
        for en in curr_entities:
            if (new_sent['start'] <= en[0] <= new_sent['end']) and (new_sent['start'] <= en[1] <= new_sent['end']):
                # add label entities
                curr_entity = ''.join(original_text[c] for c in range(en[0], en[1]))
                for ent in re.finditer(curr_entity, new_sent['text']):
                    curr_labels.append([ent.start(), ent.end(), en[2]]) # start index, end index, type
            # Add annotate sentences to annotate_sentences array
        annotated_docs.append({
            'text': new_sent['text'],
            'labels': curr_labels
        })
    # Append all sentences to concat_sent_cfg
    concat_sent_cfg += sentences_cfg

Skipping...88


In [25]:
len(annotated_docs)

6559

In [26]:
# Before pass through the POS tagger, we need to remove stop words
stop_word = ['and', 'the', 'of', 'a', 'in', 'often', 'or', 'to', 'their', 'more', 'than', 'usual', 'for', 'into', 'test']

In [27]:
'''
  {
    'text': 'Tick fever is the general name given to a group of diseases that are all caused by a tick bite', 
    'labels': [[0, 10, 'DISEASE']]
  }
'''
def format_corpus(data, idx):
    ann_tup = []
    curr_sen_en = [[''.join(data['text'][c] for c in range(en[0], en[1])), en[2]] for en in data['labels']]
    for word in data['text'].split(' '):
        en_type = 'OTHER'
        for cse in curr_sen_en:
            for sw in cse[0].split(' '):
                if sw == word:
                    en_type = cse[1]
        if str(word).strip() != '' and not word in stop_word:
            ann_tup.append((word, en_type))
    return ann_tup

## Train POS using Custom Corpus

In [28]:
word_ann_docs = []
for idx,doc in enumerate(annotated_docs):
    word_ann_docs += format_corpus(doc, idx)

In [29]:
# Helper functions to define the properties of the model
def features(sentence, index):
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }

def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
    return X, y

In [30]:
# Setup dataset as [('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ',')]
tagged_sentences = [format_corpus(sentence, idx) for idx,sentence in enumerate(annotated_docs)]

# Split the dataset for training and testing
cutoff = int(.75 * len(tagged_sentences))
training_sentences = tagged_sentences[:cutoff]
test_sentences = tagged_sentences[cutoff:]

In [31]:
X, y = transform_to_dataset(training_sentences)

In [32]:
# Train the model using the decision tree classifier to idetify the named entities
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
 
clf.fit(X[:10000], y[:10000])

Pipeline(steps=[('vectorizer', DictVectorizer(sparse=False)),
                ('classifier', DecisionTreeClassifier(criterion='entropy'))])

In [33]:
# Save model as pickel file
# sci-kit learn documentation suggest to save the model as .joblib instead of .pickle
import joblib

joblib.dump(clf, 'pos-tagger-benchmark.joblib')

['pos-tagger-benchmark.joblib']

## Tryout implementation

In [34]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [35]:
# Preprocessing (remove stop words)
user_input = "This model finds disease names such as Cholera, Cancer or COVID"

# Tokenize the sentence
tokens = nltk.word_tokenize(user_input)

# Remove stop words, (stopwords are getting fromt the NLTK library)
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

# Filtered tokens as sentence
filtered_user_input = " ".join(filtered_tokens)

In [36]:
# load model
pos_tagger = joblib.load("pos-tagger-benchmark.joblib")

In [37]:
filtered_user_input

'model finds disease names Cholera , Cancer COVID'

In [38]:
from nltk import word_tokenize, pos_tag

def pos_tag(sentence):
  print([features(sentence, index) for index in range(len(sentence))])
  tags = pos_tagger.predict([features(sentence, index) for index in range(len(sentence))])
  return zip(sentence, tags)

list(pos_tag(word_tokenize(filtered_user_input)))

[{'word': 'model', 'is_first': True, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': True, 'prefix-1': 'm', 'prefix-2': 'mo', 'prefix-3': 'mod', 'suffix-1': 'l', 'suffix-2': 'el', 'suffix-3': 'del', 'prev_word': '', 'next_word': 'finds', 'has_hyphen': False, 'is_numeric': False, 'capitals_inside': False}, {'word': 'finds', 'is_first': False, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': True, 'prefix-1': 'f', 'prefix-2': 'fi', 'prefix-3': 'fin', 'suffix-1': 's', 'suffix-2': 'ds', 'suffix-3': 'nds', 'prev_word': 'model', 'next_word': 'disease', 'has_hyphen': False, 'is_numeric': False, 'capitals_inside': False}, {'word': 'disease', 'is_first': False, 'is_last': False, 'is_capitalized': False, 'is_all_caps': False, 'is_all_lower': True, 'prefix-1': 'd', 'prefix-2': 'di', 'prefix-3': 'dis', 'suffix-1': 'e', 'suffix-2': 'se', 'suffix-3': 'ase', 'prev_word': 'finds', 'next_word': 'names', 'has_hyphen': False, 'is_numeric': 

[('model', 'OTHER'),
 ('finds', 'OTHER'),
 ('disease', 'DISEASE'),
 ('names', 'OTHER'),
 ('Cholera', 'OTHER'),
 (',', 'OTHER'),
 ('Cancer', 'OTHER'),
 ('COVID', 'DISEASE')]

## Evaluation

In [39]:
X_test, y_test = transform_to_dataset(test_sentences)
print("Accuracy:", clf.score(X_test, y_test))

Accuracy: 0.8435126101488909


In [40]:
y_true = y_test # Test data
y_pred = [pos_tagger.predict(word)[0] for word in X_test]
target_names = list(set(y_test))

In [41]:
print(target_names)

['SYMPTOM', 'SKINRAREDISEASE', 'SIGN', 'ANAPHOR', 'RAREDISEASE', 'DISEASE', 'OTHER']


In [42]:
# Perform one hot encoding

from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
encoder.fit(target_names)

actual = encoder.transform(y_true)
predict = encoder.transform(y_pred)

In [43]:
from sklearn.metrics import classification_report

print(classification_report(actual, predict, target_names=target_names))

                 precision    recall  f1-score   support

        SYMPTOM       0.81      0.75      0.78       413
SKINRAREDISEASE       0.36      0.30      0.33       529
           SIGN       0.89      0.96      0.92     18518
        ANAPHOR       0.61      0.41      0.49      1474
    RAREDISEASE       0.46      0.29      0.35      1826
        DISEASE       0.17      0.05      0.08       144
          OTHER       0.93      0.21      0.34       133

       accuracy                           0.84     23037
      macro avg       0.61      0.43      0.47     23037
   weighted avg       0.82      0.84      0.83     23037

